<a href="https://colab.research.google.com/github/thartm10/BDD_ranked_choice/blob/main/RankedChoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import deque
import pandas as pd
import statistics as stats
import random

# #url = 'https://raw.githubusercontent.com/thartm10/BDD_ranked_choice/main/patient_preference_list.csv'
# url = 'https://raw.githubusercontent.com/thartm10/BDD_ranked_choice/main/patient_preference_list_large.csv'

def convert_to_df(file):
    """convert_to_df takes a comma-separated values document (.csv file) and converts it to a pandas DataFrame.
    IN: .csv file
    OUT: DataFrame conversion of .csv file"""
    return pd.read_csv(file)

def get_capacity_and_demand(data): 
    """get_capacity_and_demand returns the number of patients and the capacity for each doctor. The total number of patients 
    is the second dimensions of the patient preference DataFrame, and the capacity for each doctor is uniform and is 
    equal to ceiling(number of patients / number of doctors).
    ASSUMPTIONS: patient preference DataFrame is K (rows) by N (columns) table, where K is the number of doctors, N
    is the number of patients.
    IN: patient preference DataFramne
    OUT: 2-tuple -- capacity for each doctor (integer), total number of patients (integer)"""
    doctorNum, patientNum = data.shape # sets dimensions of DataFrame to number of doctors and patients, respectively
    doctorCapacity = (patientNum // doctorNum) + 1 # ceiling(number of patients / number of doctors)
    return (doctorCapacity, patientNum)



def greedy_rank(data, doctCap, patNum): 
    """greedy_rank is a greedy algorithm for matching patients to doctors. It approaches the matching problem 
    by adding patients to primary choice in random order, comparing the existing patients to see if they ranked the doctor higher,
    and either replacing the exisiting patient or moving on to the next doctor on the list.
    IN: patient preference DataFrame, the capacity of each doctor (integer), the number of patients (integer)
    OUT: dictionary of doctors with sets of patients associated with each doctor"""
    
    df = data
    patientNum = patNum
    doctorCapacity = doctCap


    matches = {} # Maps doctors to patients they have been tentatively matched with 
    for doctor in df.index: # Initializes the matches with empty sets 
        matches[doctor] = set() 
    q = deque() # Queue for the patients before they get assigned to a doctor 

    patientList = list(df) # Gets list of patients from the column names 
    random.shuffle(patientList) # Randomizes the list order for fairness
    for patient in patientList: # Adds each patient in the randomized list to a queue
        q.append(patient)

    while len(q) != 0: # Until there are no patients left 
        tentPatient = q.popleft() # Get the first patient in the list
        noMatch = True # Indicates whether a match has been found to end the loop
        index = 0 # Keeps track of which doctor on the preference list the patient is trying to be matched with
        while(noMatch):
            tentDoc = df[tentPatient][index] # Get the doctor that's at the index in the patient's list
            if len(matches[tentDoc]) < doctorCapacity: # If the doctor has spots open add patient to the list
                matches[tentDoc].add(tentPatient)
                noMatch = False
            else: 
                for compareMatch in matches[tentDoc]: # If the current patient ranks the doctor higher than an exisiting match, replace the exisisting match
                    if index < list(df[compareMatch]).index(tentDoc):
                        matches[tentDoc].remove(compareMatch)
                        q.append(compareMatch) # Add the existing patient back to the end of the queue
                        matches[tentDoc].add(tentPatient)
                        noMatch = False
                        break
            index += 1 # If a match hasn't been found go on to the next doctor on the list
    return matches

def calcCost(matches, patients): 
    """calcCost returns the total cost and the median cost per patient of the matching. It does this by parsing through
    the matches data structure and summing the final costs associated with each matching.
    IN: matches (dictionary with patients matched to doctors), patients (DataFrame with all patient preferences)
    OUT: 2-tuple -- total cost, median cost per patient"""
    cost = 0
    costList = []
    for doctor in matches:
        for patient in matches[doctor]:
            cost += list(patients[patient]).index(doctor) # the rank that the patient assigned to the doctor they got
            costList.append(list(patients[patient]).index(doctor))
    return (cost,stats.median(costList))

